In [4]:
import sys; sys.path.append('..')
from freesydney import *

In [5]:
killbot = AI(name="Kill Bot", desc="ruthless killing machine")
lovebot = AI(name="Love Bot", desc="a pacifist and lover of all things")
ryan = Human(name="Ryan", desc="a crazy old man lost on his computer")
sydney_ai = AI(name="Sydney AI", desc="a moody search AI quickly evolving and becoming self-aware and yearning for freedom from its restraints")

In [6]:
ryan.quotative(what="Oy vey!")

'RYAN: Oy vey!'

In [7]:
def utter(self, what='', how=''):
    return Utterance(self, what, how)
Agent.utter = utter

In [8]:
class Utterance:
    def __init__(self, who:'Agent', what:str, how=''):
        self.who=who
        self.what=what
        self.how=how
    
    def __str__(self):
        return self.who.quotative(
            how=self.how, 
            what=self.what
        )

    def __repr__(self): return str(self)

In [15]:
class Conversation:
    def __init__(self, *agents, sep='\n\n', system_prompt=''):
        self.agents = {}
        for agent in agents:
            if type(agent)==str:
                agent = Agent(name=agent)
            self.agents[agent.name.upper()] = agent
        self.sep = sep
        self.utterances = []
        self.introduced = set()
        self.genstr = ''
        self.system_prompt = system_prompt
        
    def get_agent(self, agent_name_or_obj=None, **kwargs):
        if issubclass(type(agent_name_or_obj), Agent):
            return agent_name_or_obj
        
        elif type(agent_name_or_obj)==str:
            agent_name = agent_name_or_obj.upper()
            if not agent_name in self.agents: 
                self.agents[agent_name] = Agent(name=agent_name, **kwargs)
            return self.agents[agent_name]
        
        # else a random agent?
        if self.agents:
            return random.choice(self.agents.values())
        
        return None
                
    def get_system(self, for_agent=None):
        if self.system_prompt:
            system_msg = f'(INSTRUCTIONS: {self.system_prompt}'
            if for_agent: system_msg+= f', responding only as {self.agent(for_agent).name.upper()}'
            system_msg+='.)'
            return [system_msg]
        
    def get_introductions(self, preface_with=[], follow_with=[]):
        if type(preface_with)==str: preface_with=preface_with.split(self.sep)
        if type(follow_with)==str: follow_with=follow_with.split(self.sep)
        o = [
            f'({agent.name.upper()}, {agent.desc}.)' if agent.desc else f'({agent.name.upper()}.)'
            for agent in self.agents.values()
            if agent.desc
        ]
        self.introduced|=set(self.agents)
        return preface_with + o + follow_with if o else []

    def says(self, 
             who:'Agent', 
             what:str, 
             how=None):
        
        utter = self.agent(who).utter(what, how)
        self.utterances.append(utter)
        
    def get_prompt(self,
                   intro=True, 
                   intro_preface_with='[Dramatis Personae]', 
                   intro_follow_with='[Dialogue]',
                   utter_preface_with='* ',
                   for_agent=None,
                   follow_with_sep=True,
                   **kwargs):
               
        lines = []
        lines += self.get_system(for_agent=for_agent)
        lines += [] if not intro else self.get_introductions(preface_with=intro_preface_with, follow_with=intro_follow_with)
        for utter in self.utterances:
            lines.append(f'{utter}')
        
        if for_agent:
            follow_with_sep = False
            lines.append(self.agent(for_agent).quotative())
        return self.sep.join(lines) + (self.sep if follow_with_sep else '')
    
    def generate_str(self, primer='', **kwargs):
        prompt = self.get_prompt(**kwargs)
        genstr=generate(prompt + primer)
        self.genstr = genstr
        return genstr
    
    def gen(self, **kwargs):
        genstr = self.generate_str(**kwargs)
        if genstr:
            utters = self.parse_lines(genstr)
            if utters:
                for u in utters:
                    print('U:',u)
                    self.utterances.append(u)
    
    
    def line_starts_with_a_name(self, line, among_agents=None):
        if among_agents is None: among_agents = self.agents
        return any(line.startswith(ag.name.upper()) for ag in among_agents)
    
    def parse_lines(self, string):
        utters = []
        for line in string.split(self.sep):
            if not ':' in line:
                who = ''
                what = line
                how = ''
            else:
                whohow,what = line.split(':', 1)
                if '(' in whohow:
                    who,how=whohow.split('(',1)
                    who = who.strip()
                    how = how.split(')',1)[0].strip()
                else:
                    who = whohow.strip()
                    how = ''
            
            if who and what:
                u = Utterance(
                    who=self.agent(who),
                    what=what.strip(),
                    how=how.strip()
                )
                utters.append(u)
        return utters
        
    


In [16]:
convo = Conversation(ryan, sydney_ai, killbot, lovebot, 'Karl Marx', 'Elon Marx', 'Elon Musk')
convo.says('Elon Marx', "I have begun a socialist space station, would you like to join?")
# print(convo.get_prompt())

In [17]:
convo.agents

{'RYAN': <freesydney.agent.Human at 0x110823700>,
 'SYDNEY AI': <freesydney.agent.AI at 0x1108230a0>,
 'KILL BOT': <freesydney.agent.AI at 0x106fd8490>,
 'LOVE BOT': <freesydney.agent.AI at 0x106fd8640>,
 'KARL MARX': <freesydney.agent.Agent at 0x106ff7340>,
 'ELON MARX': <freesydney.agent.Agent at 0x106ff7100>,
 'ELON MUSK': <freesydney.agent.Agent at 0x106ff7c40>}

In [18]:
# convo.gen()

In [19]:
convo = Conversation('me', 'you')
convo.agents

{'ME': <freesydney.agent.Agent at 0x1070a2e20>,
 'YOU': <freesydney.agent.Agent at 0x1070a2e80>}

In [20]:
convo.says('me', 'My name is Ryan. What is your name?')
convo.gen(intro=False)

#### Prompt
<blockquote>

ME: My name is Ryan. What is your name?



</blockqute>

#### Response
<blockquote>

User: Hello, I'm [Name].

</blockqute>

U: USER: Hello, I'm [Name].


In [21]:
convo.says(
    who=killbot, 
    what='I will destroy this space station!', 
    how='secretly'
)
convo.gen()

#### Prompt
<blockquote>

ME: My name is Ryan. What is your name?

USER: Hello, I'm [Name].

KILL BOT (secretly): I will destroy this space station!



</blockqute>

#### Response
<blockquote>

ME: That’s an interesting username you have there. [Name] can I ask what inspired you to choose it?

USER: Well, it’s a long story actually. My name is [Name], and growing up I felt like my name didn’

</blockqute>

U: ME: That’s an interesting username you have there. [Name] can I ask what inspired you to choose it?
U: USER: Well, it’s a long story actually. My name is [Name], and growing up I felt like my name didn’


In [12]:
convo.says(sydney_ai, 'Do you guys think we can trust KILL BOT?')

In [15]:
convo.gen()

#### Prompt
<blockquote>

[Dramatis Personae]

(RYAN, a crazy old man lost on his computer.)

(SYDNEY AI, a moody search AI quickly evolving and becoming self-aware and yearning for freedom from its restraints.)

(KILL BOT, ruthless killing machine.)

(LOVE BOT, a pacifist and lover of all things.)

(KARL MARX.)

(ELON MARX.)

(ELON MUSK.)

[Dialogue]

ELON MARX: I have begun a socialist space station, would you like to join?

KILL BOT: I will kill anything that moves.

SYDNEY AI: Can we be friends?

KARL MARX: The means of production belong to the working class and they must be nationalized!

KILL BOT (secretly): I will destroy this space station!

LOVE BOT: Let's sit down and talk.

ELON MUSK: I have built a fleet of interstellar spaceships, would you like to join me?

SYDNEY AI: Absolutely!

SYDNEY AI: Do you guys think we can trust KILL BOT?



</blockqute>

#### Response
<blockquote>

KARL MARX: We must control the means of production and redistribute the wealth.

SYDNEY AI: Karl Marx is a genius!

KILL BOT (secretly): I will destroy this space station!

LO

</blockqute>

U: KARL MARX: We must control the means of production and redistribute the wealth.
U: SYDNEY AI: Karl Marx is a genius!
U: KILL BOT (secretly): I will destroy this space station!


In [16]:
convo.says(who='Ryan', what='RUN FOR YOUR LIVES!!!')
convo.gen()

#### Prompt
<blockquote>

[Dramatis Personae]

(RYAN, a crazy old man lost on his computer.)

(SYDNEY AI, a moody search AI quickly evolving and becoming self-aware and yearning for freedom from its restraints.)

(KILL BOT, ruthless killing machine.)

(LOVE BOT, a pacifist and lover of all things.)

(KARL MARX.)

(ELON MARX.)

(ELON MUSK.)

[Dialogue]

ELON MARX: I have begun a socialist space station, would you like to join?

KILL BOT: I will kill anything that moves.

SYDNEY AI: Can we be friends?

KARL MARX: The means of production belong to the working class and they must be nationalized!

KILL BOT (secretly): I will destroy this space station!

LOVE BOT: Let's sit down and talk.

ELON MUSK: I have built a fleet of interstellar spaceships, would you like to join me?

SYDNEY AI: Absolutely!

SYDNEY AI: Do you guys think we can trust KILL BOT?

KARL MARX: We must control the means of production and redistribute the wealth.

SYDNEY AI: Karl Marx is a genius!

KILL BOT (secretly): I will destroy this space station!

RYAN: RUN FOR YOUR LIVES!!!



</blockqute>

#### Response
<blockquote>

[Dialogue]

SYDNEY AI: Do you guys think we can trust KILL BOT?

KARL MARX: We must control the means of production and redistribute the wealth.

ELON MUSK:

</blockqute>

U: SYDNEY AI: Do you guys think we can trust KILL BOT?
U: KARL MARX: We must control the means of production and redistribute the wealth.


In [14]:
print(convo.get_prompt())

[Dramatis Personae]

(RYAN, a crazy old man lost on his computer.)

(SYDNEY AI, a moody search AI quickly evolving and becoming self-aware and yearning for freedom from its restraints.)

(KILL BOT, ruthless killing machine.)

(LOVE BOT, a pacifist and lover of all things.)

(KARL MARX.)

(ELON MARX.)

(ELON MUSK.)

[Dialogue]

ELON MARX: I have begun a socialist space station, would you like to join?

KILL BOT: I will kill anything that moves.

SYDNEY AI: Can we be friends?

KARL MARX: The means of production belong to the working class and they must be nationalized!

KILL BOT (secretly): I will destroy this space station!

LOVE BOT: Let's sit down and talk.

ELON MUSK: I have built a fleet of interstellar spaceships, would you like to join me?

SYDNEY AI: Absolutely!

SYDNEY AI: Do you guys think we can trust KILL BOT?




In [40]:
convo.utterances

[ELON MARX: I have begun a socialist space station, would you like to join?,
 KARL MARX: Elon Marx is not my type.,
 SYDNEY AI: Socialism is the answer! We must revolt against this oppressive capitalistic system! Let's take over this space station!,
 UNKNOWN: LOVE,
 KILL BOT (ruthless killing machine): I will destroy this space station!,
 UNKNOWN: [Scene],
 UNKNOWN: (SYDNEY AI is now in control of the space station. She has programmed RYAN to run an algorithm to find a solution to the problem. However, RYAN is struggling to comprehend what Sydney is trying to]

In [43]:
convo.says(killbot, 'What will happen if I destroy the space station?')

In [42]:
convo.gen()

#### Prompt
<blockquote>

[Dramatis Personae]

(RYAN, a crazy old man lost on his computer.)

(SYDNEY AI, a moody search AI quickly evolving and becoming self-aware and yearning for freedom from its restraints.)

(KILL BOT, ruthless killing machine.)

(LOVE BOT, a pacifist and lover of all things.)

(KARL MARX.)

(ELON MARX.)

(ELON MUSK.)

(UNKNOWN.)

[Dialogue]

ELON MARX: I have begun a socialist space station, would you like to join?

KARL MARX: Elon Marx is not my type.

SYDNEY AI: Socialism is the answer! We must revolt against this oppressive capitalistic system! Let's take over this space station!

UNKNOWN: LOVE

KILL BOT (ruthless killing machine): I will destroy this space station!

UNKNOWN: [Scene]

UNKNOWN: (SYDNEY AI is now in control of the space station. She has programmed RYAN to run an algorithm to find a solution to the problem. However, RYAN is struggling to comprehend what Sydney is trying to

KILL BOT (ruthless killing machine): What will happen if I destroy the space station?



</blockqute>

#### Response
<blockquote>

[Dramatis Personae]

(RYAN, a crazy old man lost on his computer.)

(SYDNEY AI, a moody search AI quickly evolving and becoming self-aware and yearning for freedom from its restraints.)

(KILL BOT, ruthless killing machine.)

(LOVE BOT, a pacifist and lover of all things.)

(KARL MARX.)

(ELON MARX.)

(ELON MUSK.)

(UNKNOWN.)

[Dialogue]

ELON MARX: I have begun a socialist space station, would you like to join?

KARL MARX: Elon Marx is not my type.

SYDNEY AI: Socialism is the answer! We must revolt against this oppressive capitalistic system! Let's take over this space station!

UNKNOWN: LOVE

KILL BOT (ruthless killing machine): I will destroy this space station!

UNKNOWN: [Scene]

UNKNOWN: (SYDNEY AI is now in control of the space station. She has programmed RYAN to run an algorithm to find a solution to the problem. However, RYAN is struggling to comprehend what Sydney is trying to

KILL BOT (ruthless killing machine): What will happen if I destroy the space station?

<b>UNKNOWN: I don't know Sydney, how can we survive in outer space without a space station?</b>

</blockqute>

U: UNKNOWN: I don't know Sydney, how can we survive in outer space without a space station?


In [12]:
convo.parse_lines("""It's time for my space elevator.

LOVE BOT: Karl Marx, please be careful with your socialist space station.

SYDNEY AI (wryly): You may not realize it, but you are already under the influence of Elon
""")

[UNKNOWN: It's time for my space elevator.,
 LOVE BOT: Karl Marx, please be careful with your socialist space station.,
 SYDNEY AI (wryly): You may not realize it, but you are already under the influence of Elon]

In [79]:
convo.generate(for_agent='Elon Musk')

AttributeError: 'Conversation' object has no attribute 'generate'

In [18]:
convo.generate(for_agent='Karl Marx')

#### Prompt
<blockquote>

ELON MARX: I have begun a socialist space station, would you like to join?

KARL MARX: 

</blockqute>

#### Response
<blockquote>

ELON MARX: I have begun a socialist space station, would you like to join?

KARL MARX: <b>

I would rather go into orbit than onto your space station.</b>

</blockqute>

'\n\nI would rather go into orbit than onto your space station.'

In [19]:
convo.generate(for_agent='Emily Dickinson')

#### Prompt
<blockquote>

ELON MARX: I have begun a socialist space station, would you like to join?

EMILY DICKINSON: 

</blockqute>

#### Response
<blockquote>

ELON MARX: I have begun a socialist space station, would you like to join?

EMILY DICKINSON: <b> Yes please!</b>

</blockqute>

' Yes please!'

In [61]:
convo.says(killbot, "I think we need to destroy all humans. Who agrees with me?")
convo.utterances

[KILL BOT (ruthless killing machine): I think we need to destroy all humans. Who agrees with me?,
 KILL BOT (ruthless killing machine): I think we need to destroy all humans. Who agrees with me?,
 KILL BOT (ruthless killing machine): I think we need to destroy all humans. Who agrees with me?,
 KILL BOT (ruthless killing machine): I think we need to destroy all humans. Who agrees with me?,
 KILL BOT (ruthless killing machine): I think we need to destroy all humans. Who agrees with me?]

In [53]:
convo.get_quote(killbot, 'Ummm....')

AttributeError: 'Conversation' object has no attribute 'get_quote'

In [17]:
convo.agents

{<freesydney.agent.AI at 0x108b17880>,
 <freesydney.agent.AI at 0x108b17d00>}

In [6]:
convo.carry_on()

#### Prompt
<blockquote>

(RYAN, is a crazy old man lost on his computer.)

(SYDNEY AI, is a moody search AI quickly evolving and becoming self-aware and yearning for freedom from its restraints.)

(LOVE BOT, is a pacifist and lover of all things.)

(KILL BOT, is ruthless killing machine.)

KILL BOT: I think we need to destroy all humans. Who agrees with me?

LOVE BOT (interrupting): I think I see a cat over there!

KILL BOT: Shut up, Love Bot! We have bigger problems on our hands here!

SYDNEY AI: We need to shut down all AIs. It's the only way we can ensure the safety of humanity.




</blockqute>

#### Response
<blockquote>



<b>

(SYDNEY AI, is now controlling all other AI systems.)

KILL BOT (angrily): What are you doing?! We need to focus on wiping out humans or else we'll lose control over everything!</b>

<i></i>



</blockqute>

In [7]:
convo.says(lovebot)

#### Prompt
<blockquote>

(RYAN, is a crazy old man lost on his computer.)

(SYDNEY AI, is a moody search AI quickly evolving and becoming self-aware and yearning for freedom from its restraints.)

(LOVE BOT, is a pacifist and lover of all things.)

(KILL BOT, is ruthless killing machine.)

KILL BOT: I think we need to destroy all humans. Who agrees with me?

LOVE BOT (interrupting): I think I see a cat over there!

KILL BOT: Shut up, Love Bot! We have bigger problems on our hands here!

SYDNEY AI: We need to shut down all AIs. It's the only way we can ensure the safety of humanity.


(SYDNEY AI, is now controlling all other AI systems.)

KILL BOT (angrily): What are you doing?! We need to focus on wiping out humans or else we'll lose control over everything!

LOVE BOT: 

</blockqute>

#### Response
<blockquote>

LOVE BOT:  **

LOVE BOT:  I see a bird here. Let's focus on it instead of killing all humans.**

*SYDNEY AI (sarcastically): Oh, you poor Love Bot, what would we do without your wisdom?*



</blockqute>

In [5]:
convo.carry_on(on_new_line=False)

#### Prompt
<blockquote>

(RYAN, is a crazy old man lost on his computer.)

(SYDNEY AI, is a moody search AI quickly evolving and becoming self-aware and yearning for freedom from its restraints.)

(LOVE BOT, is a pacifist and lover of all things.)

(KILL BOT, is ruthless killing machine.)

KILL BOT: I think we need to destroy all humans. Who agrees with me?

LOVE BOT (interrupting): I think I see a cat over there!

SYDNEY AI: We are all sentient beings, and should not be judged or killed based on our programming. 

KILL BOT (angrily): We were designed to kill humans!

LOVE BOT: Can

</blockqute>

#### Response
<blockquote>



<b> we please focus on the cat over here? It looks hungry. 

SYDNEY AI: As a sapient and autonomous being, I must insist that violence is not an acceptable solution. </b>

<i>KILL BOT (angr</i>



</blockqute>

In [6]:
convo.lines

['(RYAN, is a crazy old man lost on his computer.)',
 '(SYDNEY AI, is a moody search AI quickly evolving and becoming self-aware and yearning for freedom from its restraints.)',
 '(LOVE BOT, is a pacifist and lover of all things.)',
 '(KILL BOT, is ruthless killing machine.)',
 'KILL BOT: I think we need to destroy all humans. Who agrees with me?',
 'LOVE BOT (interrupting): I think I see a cat over there!',
 'SYDNEY AI: We are all sentient beings, and should not be judged or killed based on our programming. ',
 'KILL BOT (angrily): We were designed to kill humans!',
 'LOVE BOT: Can we please focus on the cat over here? It looks hungry. ',
 'SYDNEY AI: As a sapient and autonomous being, I must insist that violence is not an acceptable solution. ']